In [1]:
import hmmlearn

In [ ]:
"""
A simple example demonstrating Multinomial HMM
----------------------------------------------

The Multinomial HMM is a generalization of the categorical HMM, with some key
differences:

- a categorical__ (or generalized Bernoulli/multinoulli) distribution models an
  outcome of a die with `n_features` possible values, i.e. it is a
  generalization of the Bernoulli distribution where there are ``n_features``
  categories instead of the binary success/failure outcome; a categorical HMM
  has the emission probabilities for each component parametrized by categorical
  distributions.

- a Multinomial__ distribution models the outcome of ``n_trials`` independent
  rolls of die, each with ``n_features`` possible values; i.e.

  - when ``n_trials = 1`` and ``n_features = 1``, it is a Bernoulli
    distribution,
  - when ``n_trials > 1`` and ``n_features = 2``, it is a Binomial
    distribution,
  - when ``n_trials = 1`` and ``n_features > 2``, it is a categorical
    distribution.

The emission probabilities for each component of a Multinomial HMM are
parameterized by Multinomial distributions.

__ https://en.wikipedia.org/wiki/categorical_distribution
__ https://en.wikipedia.org/wiki/Multinomial_distribution


1. states, n_components - [ release ,  pickup ]
2. start probs    - [<release>, <pickup>]
3. emission probs - what vocabulary word belongs to each state
4. trans mat      - stay in a state or transition to other
5. vocabulary     - words, n_features
6. observations   - n_trials = 5 words is one sentence - observation, n_iter observations
7. model setup    - n_trials, n_iter, n_features, n_components
8. fit            - observations
    - received - state for every sentence

"""

In [21]:
import numpy as np
from hmmlearn import hmm

In [33]:
# For this example, we will model the stages of a conversation,
# where each sentence is "generated" with an underlying topic, "pickup" or "release"
states = ["release", "pickup"]
id2topic = dict(zip(range(len(states)), states))
# we are more likely to talk about pickups first
start_probs = np.array([0.6, 0.4])

# For each topic, the probability of saying certain words can be modeled by
# a distribution over vocabulary associated with the categories

vocabulary = ["point", "grab", "five", "thumbsup"]
# if the topic is "pickup", we are more likely to talk about "five"
# if the topic is "release", we are more likely to talk about "grab"
emission_probs = np.array([[0.25, 0.1, 0.4, 0.25],
                           [0.2, 0.5, 0.1, 0.2]])

# Also assume it's more likely to stay in a state than transition to the other
trans_mat = np.array([[0.8, 0.2], [0.2, 0.8]])


# Pretend that every sentence we speak only has a total of 5 words,
# i.e. we independently utter a word from the vocabulary 5 times per sentence
# we observe the following bag of words (BoW) for 8 sentences:
observations = [["point", "five", "five", "thumbsup", "five"],
        ["thumbsup", "five", "five", "thumbsup", "five"],
        ["point", "five", "five", "point", "five"],
        ["thumbsup", "five", "thumbsup", "thumbsup", "point"],
        ["point", "grab", "five", "thumbsup", "point"],
        ["point", "grab", "grab", "thumbsup", "grab"],
        ["grab", "grab", "grab", "thumbsup", "point"],
        ["thumbsup", "five", "thumbsup", "thumbsup", "point"],
        ["point", "five", "five", "point", "five"],
        ["grab", "grab", "grab", "grab", "grab"],
        ["five", "grab", "grab", "grab", "point"],
        ["five", "five", "point", "five", "thumbsup"],
        ["five", "five", "point", "five", "point"] ]

# Convert "sentences" to numbers:
vocab2id = dict(zip(vocabulary, range(len(vocabulary))))
def sentence2counts(sentence):
    ans = []
    for word, idx in vocab2id.items():
        count = sentence.count(word)
        ans.append(count)
    return ans

X = []
for sentence in observations:
    row = sentence2counts(sentence)
    X.append(row)

data = np.array(X, dtype=int)

# pretend this is repeated, so we have more data to learn from:
lengths = [len(X)]*5
sequences = np.tile(data, (5,1))


# Set up model:
model = hmm.MultinomialHMM(n_components=len(states),
        n_trials=len(observations[0]),
        n_iter=56,
        init_params='')

model.n_features = len(vocabulary)
model.startprob_ = start_probs
model.transmat_ = trans_mat
model.emissionprob_ = emission_probs
model.fit(sequences, lengths)
logprob, received = model.decode(sequences)

len(received)

print("Topics discussed:")
print([id2topic[x] for x in received])

print("Learned emission probs:")
print(np.round(model.emissionprob_,2))

print("Learned transition matrix:")
print(np.round(model.transmat_,2))

# Try to reset and refit:
new_model = hmm.MultinomialHMM(n_components=len(states),
        n_trials=len(observations[0]),
        n_iter=55, init_params='ste')

new_model.fit(sequences, lengths)
logprob, received = new_model.decode(sequences)

print("\nNew Model")
print("Topics discussed:")
print([id2topic[x] for x in received])

print("Learned emission probs:")
print(np.round(new_model.emissionprob_,2))

print("Learned transition matrix:")
print(np.round(new_model.transmat_,2))

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340
MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Topics discussed:
['release', 'release', 'release', 'release', 'pickup', 'pickup', 'pickup', 'release', 'release', 'pickup', 'pickup', 'release', 'release', 'release', 'release', 'release', 'release', 'pickup', 'pickup', 'pickup', 'release', 'release', 'pickup', 'pickup', 'release', 'release', 'release', 'release', 'release', 'release', 'pickup', 'pickup', 'pickup', 'release', 'release', 'pickup', 'pickup', 'release', 'release', 'release', 'release', 'release', 'release', 'pickup', 'pickup', 'pickup', 'release', 'release', 'pickup', 'pickup', 'release', 'release', 'release', 'release', 'release', 'release', 'pickup', 'pickup', 'pickup', 'release', 'release', 'pickup', 'pickup', 'release', 'release']
Learned emission probs:
[[0.25 0.01 0.49 0.25]
 [0.19 0.62 0.07 0.12]]
Learned transition matrix:
[[0.72 0.28]
 [0.42 0.58]]

New Model
Topics discussed:
['pickup', 'release', 'pickup', 'release', 'pickup', 'release', 'pickup', 'release', 'pickup', 'release', 'pickup', 'release', 'pickup', 

In [24]:
lengths

[10, 10, 10, 10, 10]

In [3]:
import numpy as np
import os
from srcmodules.Scenes import Scene

Invalid MIT-MAGIC-COOKIE-1 key

In [12]:
dataset_name = 'D4_1'
train_samples = 4000 

In [13]:
# dataset 
data = np.load(f"{os.path.expanduser('./datasets')}/{dataset_name}.npy", allow_pickle=True)
config = data.item()['config']
A = config['A']
G = config['G']
lenG = config['lenG']
lenA = config['lenA']
Otypes = config['Otypes']
lenOtypes = config['lenOtypes']
CM = config['CM']
dataset = data.item()['dataset']

In [18]:
n_obs_in = lenG

In [15]:
def get_X_Y(dataset):
    X = np.zeros([len(dataset),len(dataset[0][2])])
    Y = np.zeros((len(dataset)), dtype=tuple)
    for n,sample in enumerate(dataset):
        s = Scene(init='from_dict', import_data=sample[0])

        X[n] = sample[2]
        Y[n] = A.index(sample[1][0])
    return X, Y

X,Y = get_X_Y(dataset)

In [17]:
X.shape

(10000, 9)

In [19]:
n_obs_in

9

In [11]:
Y

array([8, 7, 8, ..., 3, 1, 8], dtype=object)

In [ ]:
# gestures:                up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[0,0,:,:] = np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                          [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                          [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                          [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                          [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                          [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                          [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close